***Steps For Detecting Vehicles applied in Traditional Non CNN ways*** 

***Feature extraction of:***
Histogram of Oriented Gradients = 2X2 block with a 64X64 image with 9 orientations  = 7X7X9×2×2
Spatial Informatioin = resize to 32X32 and ravel
Color basic histogram = histogram of 32 bins and ravel

Linear SVM clasifier with sklearn svm  applying a supervised training set with labels.
Sliding-window used with the above trained classifier to search for vehicles in images.
Video stream creatig a heat map of recurring sequence detections and reject outliers and follow detected vehicles.

Estimate a bounding box for vehicles detected.

In [223]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import glob
import time
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.ndimage.measurements import label



##Feature Extraction

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.concatenate((color1, color2, color3),axis=0)
                         
def color_hist(img, nbins=32): 
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
     
    return hist_features

def extract_hog_features(img, orient=9, pix_per_cell=8, cell_per_block=2, vis=False, feature_vec=True):
    return hog(img, 
                orientations=orient, 
                pixels_per_cell=(pix_per_cell, pix_per_cell),
                cells_per_block=(cell_per_block, cell_per_block),
                visualize=vis, 
                feature_vector=feature_vec,
                transform_sqrt=True)

def hog_extract(feature_image):
    features = []
    for channel in range(feature_image.shape[2]):
        features.append(extract_hog_features(feature_image[:,:,channel]))
    return features

def extract_features(img, cspace='RGB'):
    if cspace != 'RGB':
        if cspace == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)      

    hog_features = np.ravel(hog_extract(feature_image))        
    spatial_features = bin_spatial(feature_image)
    hist_features = color_hist(feature_image)
    
    features = np.concatenate((spatial_features, hist_features, hog_features))

    return features
     

    
# Window Searching for cars with trained classifier    
    
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient=9, pix_per_cell=8, cell_per_block=2, 
              spatial_size=(32,32), hist_bins=32):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    ctrans_tosearch = cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2HLS)
    
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block*cell_per_block
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    hog1 = extract_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = extract_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = extract_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    detections = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
 
            
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            

            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                detection = ((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart))
                detections.append(detection)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                
    return draw_img, detections
    
    
def add_heat(heatmap, detections):
    for box in detections:
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap

def apply_threshold(heatmap, threshold):
    heatmap[heatmap <= threshold] = 0
    return heatmap

In [236]:
cars = []
notcars = []
for fl in glob.glob('./veh/vehicles_smallset/*/*.jpeg'): 
    cars.append(fl)
for fl in glob.glob('./veh/vehicles/*/*.png'): 
    cars.append(fl)
for i,fl in enumerate(glob.glob('./veh/non-vehicles_smallset/*/*.jpeg')):
    notcars.append(fl)
    if (i+1 >= len(cars)): #early-stop if there are more non-vehicle cars 
        break

images = np.ndarray((len(cars) + len(notcars), 64, 64, 3), dtype=np.float32)
labels = np.concatenate((np.ones(len(cars), dtype=np.int32), np.zeros(len(notcars), dtype=np.int32)))

for i,fl in enumerate(cars + notcars):
    img = mpimg.imread(fl)/255    
    images[i] = img
    
print(f"{len(cars)} vehicle images loaded; {len(notcars)} non-vehicle images loaded.")

9988 vehicle images loaded; 1125 non-vehicle images loaded.


In [237]:
X = []
for img in images:
    X.append(extract_features(img, cspace='HLS'))
X_Scaler = StandardScaler()
X_Scaler.fit(X)
X = X_Scaler.transform(X)

rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.1, random_state=rand_state)

svc = LinearSVC()
svc.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [244]:
def display_images_in_2_colums(images, captions=None, cmap=None):
    fig = plt.figure(figsize=(9, 5 * np.ceil(len(images) / 2.)))
    for i in range(len(images)):
        a = fig.add_subplot(np.ceil(len(images) / 2.), 2, i + 1)
        img = images[i]
        plt.imshow(img, cmap=cmap)
        if (not captions is None):
            a.set_title(captions[i])
    plt.show()
    
    
display_images = []
captions = []

ystart = 400
ystop_scale = [[656, 1.75], [600, 1.35],[int(600/1.35), 1]]


In [245]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

HEATMAP_BUFFER_SIZE = 30
HEATMAP_THRESHOLD_PER_FRAME = 4
heatmaps = []

def build_heatmap(img, ystop_scale=[[656, 1.75], [600, 1.35], [563, 1.3]]):
    heatmap = np.zeros((img.shape[0], img.shape[1]))
    for ystop, scale in ystop_scale:
        out_img, detections = find_cars(img, ystart, ystop, scale, svc, X_Scaler)
        add_heat(heatmap, detections)
    return heatmap

def draw_labeled_bboxes(img, labeled):
    for car_number in range(1, labeled[1]+1):
        nonzero = (labeled[0] == car_number).nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    return img

def process_image(img):
    result = np.copy(img)
    global heatmaps
    heatmaps.append(build_heatmap(result))
    if (len(heatmaps) > HEATMAP_BUFFER_SIZE):
        heatmaps = heatmaps[1:]
        
    acc_hm = np.zeros_like(heatmaps[0])
    for hm in heatmaps:
        acc_hm += hm
    
    apply_threshold(acc_hm, HEATMAP_THRESHOLD_PER_FRAME * len(heatmaps))
    
    labeled = label(acc_hm)
    draw_labeled_bboxes(result, labeled)
    caption = 'Cars found: {}'.format(labeled[1])
    cv2.putText(result, caption, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    
    return result

In [241]:
heatmaps = []
clip_output = 'test_video_output.mp4'
clip = VideoFileClip("test_video.mp4")
clip_processing = clip.fl_image(process_image)
%time clip_processing.write_videofile(clip_output, audio=False)

t:   0%|          | 0/38 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_video_output.mp4.
Moviepy - Writing video test_video_output.mp4



Moviepy - Done !
Moviepy - video ready test_video_output.mp4
CPU times: user 1min 22s, sys: 305 ms, total: 1min 22s
Wall time: 1min 24s
